## Grab CAM




<img src="https://frenzy86.s3.eu-west-2.amazonaws.com/python/data/blackbox.png
" width="1200">

While deep learning has facilitated unprecedented accuracy in image classification, object detection, and image segmentation, one of their biggest problems is model interpretability, a core component in model understanding and model debugging. In practice, deep learning models are treated as “black box” methods, and many times we have no reasonable idea as to: where the network is “looking” in the input image, Which series of neurons activated in the forward-pass during inference/prediction. How the network arrived at its final output that raises an interesting question — how can you trust the decisions of a model if you cannot properly validate how it arrived there?

<img src="https://frenzy86.s3.eu-west-2.amazonaws.com/python/grad.pn
" width="1200">

To help deep learning practitioners visually debug their models and properly understand where it’s “looking” in an image, Selvaraju et al. created Gradient-weighted Class Activation Mapping, or more simply, Grad-CAM:

Grad-CAM uses the gradients of any target concept (say logits for “dog” or even a caption), flowing into the final convolutional layer to produce a coarse localization map highlighting the important regions in the image for predicting the concept.”

Using Grad-CAM, we can visually validate where our network is looking, verifying that it is indeed looking at the correct patterns in the image and activating around those patterns.
If the network is not activating around the proper patterns/objects in the image, then we know: our network hasn’t properly learned the underlying patterns in our dataset
Our training procedure needs to be revisited
We may need to collect additional data
And most importantly, our model is not ready for deployment.
Grad-CAM is a tool that should be in any deep learning practitioner’s toolbox — take the time to learn how to apply it now.

In [1]:
# Setup for Colab
import sys
from pathlib import Path
IN_COLAB = 'google.colab' in sys.modules

srcpath = Path("src")

if IN_COLAB and not srcpath.exists(): # if running in Colab --> download src if not exists
    !apt-get install subversion
    !svn checkout https://github.com/nguyenhoa93/GradCAM_and_GuidedGradCAM_tf2/trunk/src 
    !svn checkout https://github.com/nguyenhoa93/GradCAM_and_GuidedGradCAM_tf2/trunk/assets

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1
Suggested packages:
  db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 subversion
0 upgraded, 5 newly installed, 0 to remove and 4 not upgraded.
Need to get 2,235 kB of archives.
After this operation, 9,916 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libapr1 amd64 1.6.3-2 [90.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libaprutil1 amd64 1.6.1-2 [84.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libserf-1-1 amd64 1.3.9-6 [44.4 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates/univer

In [2]:
from IPython.display import display, Javascript, HTML, clear_output, IFrame
from ipywidgets import interact, interactive, fixed, interact_manual, AppLayout, GridspecLayout
import ipywidgets as widgets
from src.gradcam import GradCAM, overlay_gradCAM
from src.guidedBackprop import GuidedBackprop, deprocess_image
from src.utils import preprocess, predict, SAMPLE_DIR, array2bytes, DECODE, INV_MAP
from src.models import load_ResNet50PlusFC, load_VanilaResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os, cv2
import matplotlib.pyplot as plt

# Header
header = widgets.HTML('<font color="#1f77b4" face="sans-serif"><center><h1>DEMO GradCAM and Guided GradCAM</h1></center></font>',
                      layout=widgets.Layout(height='auto'))
# Logo
logo = widgets.Image(
    value=open("./assets/illustrations/VietAIlogo.png", "rb").read(),
    format='png',
    width='auto',
    height='auto',
    align="center-align"
)

# Dropdowns
def on_change_im(change):
    if change['type'] == "change" and change["name"] == "value":
        img = cv2.imread(os.path.join(SAMPLE_DIR,change["new"]))
        im_arr = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        logo = widgets.Image(
            value=array2bytes(im_arr),
            format='png',
            width='auto',
            height='auto',
            align="center-align"
        )
        grid[5:13, 1:9] = widgets.HTML("")
        grid[5:13, 11:19] = logo
        grid[5:13, 21:29] = widgets.HTML("")
def on_change_model(change):
    if change['type'] == "change" and change["name"] == "value":
        chosen_model =  widgets.HTML("<center><p>Model %s loaded.<center>" % change["new"])
        grid[2,:8] = chosen_model
        
im_ls = ["--Select"] + [x for x in os.listdir("./assets/samples") if not x.startswith(".")]
im_ls.sort()
model_ls = ["--Select","VanilaResNet50", "ResNet50PlusFC"]
class_ls = ["--Select","Cat", "Dog"]
models = widgets.Dropdown(options=model_ls, description="Model",layout={'width':'auto'}, disabled=False)
imgs = widgets.Dropdown(options=im_ls, description="Image", layout={'width':'auto'}, disabled=False)
classes = widgets.Dropdown(options=class_ls, description="Class", layout={'width':'auto'}, disabled=False)
imgs.observe(on_change_im)
models.observe(on_change_model)
# Notes
# note = widgets.HTML("<p><b>Modes:</b><br>- VanilaResNet50 is retrained ResNet50 model on Dog vs. Cat dataset<br>"
#                     "- ResNet50PlusFC is the model with 2 FC layers added to ResNet50, retrained on Dog vs. Cat dataset"
#                     "<br><b>Classes: </b>If not specified, GradCAM & Guided GradCAM will be calculated based on the predicted class</p>")

# button
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style,
                          layout=widgets.Layout(height='auto', width='auto'))
pred_but = create_expanded_button("Show","info")

# Layouts
grid = GridspecLayout(20, 30, height='700px')
grid[0,:] = header
grid[1,:8] = models
grid[1,8:17] = imgs
grid[1,17:24] = classes
grid[1,25:] = pred_but
grid[5:13,11:19] = logo
# grid[17:,:] = note
display(grid)

def showCAMs(img, x, GradCAM, GuidedBP, chosen_class, upsample_size):
    cam3 = GradCAM.compute_heatmap(image=x, classIdx=chosen_class, upsample_size=upsample_size)
    gradcam = overlay_gradCAM(img, cam3)
    gradcam = cv2.cvtColor(gradcam, cv2.COLOR_BGR2RGB)
    # Guided backprop
    gb = GuidedBP.guided_backprop(x, upsample_size)
    gb_im = deprocess_image(gb)
    gb_im = cv2.cvtColor(gb_im, cv2.COLOR_BGR2RGB)
    # Guided GradCAM
    guided_gradcam = deprocess_image(gb*cam3)
    guided_gradcam = cv2.cvtColor(guided_gradcam, cv2.COLOR_BGR2RGB)
    
    # Display
    gc = widgets.Image(
            value=array2bytes(gradcam),
            format='png',
            width='auto',
            height='auto',
            align="center-align"
        )
    gbim = widgets.Image(
            value=array2bytes(gb_im),
            format='png',
            width='auto',
            height='auto',
            align="center-align"
        )
    ggc = widgets.Image(
            value=array2bytes(guided_gradcam),
            format='png',
            width='auto',
            height='auto',
            align="center-align"
        )
    grid[4, 1:9] = widgets.HTML('<center><b>GradCAM</b></center>')
    grid[4, 11:19] = widgets.HTML('<center><b>Guided Bacpropagation</b></center>')
    grid[4, 21:29] = widgets.HTML('<center><b>Guided GradCAM</b></center>')
    grid[5:13, 1:9] = gc
    grid[5:13, 11:19] = gbim
    grid[5:13, 21:29] = ggc
    
def check_button(sender):
#     if models.value == "VanilaResNet50"
    if models.value == "VanilaResNet50":
        model = load_VanilaResNet50()
        gradCAM = GradCAM(model=model, layerName="conv5_block3_out")
        guidedBP = GuidedBackprop(model=model,layerName="conv5_block3_out")
    elif models.value == "ResNet50PlusFC":
        model = load_ResNet50PlusFC()
        gradCAM = GradCAM(model=model, layerName="conv5_block3_out")
        guidedBP = GuidedBackprop(model=model, layerName="conv5_block3_out")
#     img = img_to_array(load_img(os.path.join(SAMPLE_DIR,imgs.value), target_size=(224,224)))
    img = cv2.imread(os.path.join(SAMPLE_DIR,imgs.value))
    upsample_size = (img.shape[1],img.shape[0])
    x = preprocess(imgs.value)
    pred, prob = predict(model,x)
    if classes.value == "--Select":
        classIdx = pred
    else:
        classIdx = INV_MAP[classes.value]
    
    grid[2,9:18] = widgets.HTML("<center><span>Predicted: <b>{}<b><span><center>".format(DECODE[pred]))
        
    showCAMs(img, x, gradCAM, guidedBP, classIdx, upsample_size)

pred_but.on_click(check_button)

GridspecLayout(children=(HTML(value='<font color="#1f77b4" face="sans-serif"><center><h1>DEMO GradCAM and Guid…

1/1 [==============================] - 1s 1s/step
